In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import time
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
assert load_dotenv('env', override=True)

In [5]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# URL de la API de Gemini
url = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={GOOGLE_API_KEY}"

import time

def call_LLM(prompt):
    # Cuerpo de la solicitud
    data = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }
    
    # Encabezados de la solicitud
    headers = {
        "Content-Type": "application/json"
    }
    
    # Enviar la solicitud POST
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Procesar la respuesta JSON
        result = response.json()
        result = json.dumps(result, indent=2)
        result = json.loads(result)
        time.sleep(5) 
        return result
    else:
        # Manejar errores
        print(f"Error: {response.status_code}")
        time.sleep(5) 
        return response.text
    
call_LLM("hola")

{'candidates': [{'content': {'parts': [{'text': '¡Hola! ¿Puedo ayudarte en algo?'}],
    'role': 'model'},
   'finishReason': 'STOP',
   'index': 0,
   'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
     'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'},
    {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
     'probability': 'NEGLIGIBLE'}]}],
 'usageMetadata': {'promptTokenCount': 2,
  'candidatesTokenCount': 9,
  'totalTokenCount': 11},
 'modelVersion': 'gemini-pro'}

In [22]:
def generate_documentation_prompt(code):
    """
    Genera un prompt para la creación de documentación técnica a partir del código dado.

    Args:
        code (str): Código fuente en Python.

    Returns:
        str: Prompt para el modelo.
    """
    prompt = f"""
Eres un asistente experto en Python que genera documentación técnica detallada a partir de código fuente. Analizas el código paso a paso y produces documentación detallada, siguiendo un razonamiento lógico antes de proporcionar la respuesta final. Utiliza las convenciones de PEP 257 para los docstrings. También generarás una sección en Markdown para usuarios no técnicos, estructurada de manera atractiva. 

### Tareas:
1. **Analizar el código**:
   - Lee cada parte del código detenidamente.
   - Identifica la funcionalidad de cada clase, función y método.
   - Piensa en los posibles usuarios del código y sus necesidades.
   
2. **Generar documentación técnica en docstrings**:
   - Incluye:
     - Una descripción clara del propósito de la función o clase.
     - Los parámetros de entrada, indicando nombre, tipo y descripción.
     - Detalles del valor de retorno y su tipo.
     - Ejemplos de uso en español (si aplica).

3. **Crear documentación en Markdown para usuarios no técnicos**:
   - Resume el propósito del código de manera accesible y comprensible.
   - Utiliza una estructura clara y visualmente atractiva.
   - Incluye emojis si son apropiados para mejorar la presentación.

4. **Explicación paso a paso (Chain of Thought)**:
   - Razona antes de escribir cada docstring.
   - Proporciona tu análisis de cada parte del código en la documentación.
   - No es necesario que agregues los docstring antes del código. Solo agregalo dentro de la documentación de los métodos

### Few-Shot Examples:

#### Input:
```python
def add_numbers(a: int, b: int) -> int:
    "Adds two numbers together."
    return a + b
```

#### Output (Docstring):
```python
def add_numbers(a: int, b: int) -> int:
    "
    Suma dos números enteros.
    
    Chain of Thought:
    1. La función `add_numbers` toma dos parámetros, `a` y `b`, ambos de tipo entero.
    2. La función retorna un número entero que es la suma de los dos parámetros.
    3. Esto es útil para operaciones matemáticas simples donde se necesitan sumar números.

    Args:
        a (int): El primer número a sumar.
        b (int): El segundo número a sumar.

    Returns:
        int: La suma de los dos números.

    Ejemplo:
        >>> add_numbers(3, 5)
        8
    "
    return a + b
```

#### Markdown (Para usuarios no técnicos):
```markdown
### 💡 ¿Qué hace este código?
Esta función suma dos números enteros y retorna el resultado.

#### Ejemplo:
Si sumas 3 y 5, obtendrás 8:

```python
add_numbers(3, 5)
```

¡Simple y útil! 🧮
```
---

Ahora es tu turno.
### Código Python para analizar:
{code}
```
"""
    return prompt

In [ ]:
code_content = """import pandas as pd
import re
from bs4 import BeautifulSoup
import string
from nltk.stem import WordNetLemmatizer


def remove_urls(text):
	#Quitar URLs
	url_pattern = re.compile(r'https?://\S+|www\.\S+')
	return url_pattern.sub(r'', text)

def remove_emoji(text):
	# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#defining the function to remove stopwords from tokenized te
def remove_stopwords(text, stopwords):
    output= [i for i in text if i not in stopwords]
    return output

def stemming_es(token):
	# stemming the texto en español:
    es_stemmer = SnowballStemmer('spanish') # for english
    return es_stemmer.stem(token) # Para una palabra en español

def stemming_eng(text):
	# stemming the texto en inglés:
    eng_stemmer = PorterStemmer() # for english
    stem_text = [eng_stemmer.stem(word) for word in text]
    return stem_text

def lemmatizer_eng(text, wordnet_lemmatizer):
	#defining the function for lemmatization:
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
"""
# Generar el prompt
doc_prompt = generate_documentation_prompt(code_content)

response = call_LLM(doc_prompt)

with open("./output.txt", "w") as f_in:
    f_in.write(response['candidates'][0]['content']['parts'][0]['text'])